In [1]:
import pandas as pd
dados=pd.read_csv('C:/Users/Falcone/Documents/0_MachineLearning/2022.05.23_AdaBoost/recipeData.csv', encoding='ISO-8859-1')

In [2]:
dados

,BeerID,Name,URL,Style,StyleID,Size(L),OG,FG,ABV,IBU,...,BoilGravity,Efficiency,MashThickness,SugarScale,BrewMethod,PitchRate,PrimaryTemp,PrimingMethod,PrimingAmount,UserId
0,1,Vanilla Cream Ale,/homebrew/recipe/view/1633/vanilla-cream-ale,Cream Ale,45,21.77,1.055,1.013,5.48,17.65,...,1.038,70.0,NaN,Specific Gravity,All Grain,NaN,17.78,corn sugar,4.5 oz,116.0
1,2,Southern Tier Pumking clone,/homebrew/recipe/view/16367/southern-tier-pumk...,Holiday/Winter Special Spiced Beer,85,20.82,1.083,1.021,8.16,60.65,...,1.070,70.0,NaN,Specific Gravity,All Grain,NaN,NaN,NaN,NaN,955.0
2,3,Zombie Dust Clone - EXTRACT,/homebrew/recipe/view/5920/zombie-dust-clone-e...,American IPA,7,18.93,1.063,1.018,5.91,59.25,...,NaN,70.0,NaN,Specific Gravity,extract,NaN,NaN,NaN,NaN,NaN
3,4,Zombie Dust Clone - ALL GRAIN,/homebrew/recipe/view/5916/zombie-dust-clone-a...,American IPA,7,22.71,1.061,1.017,5.80,54.48,...,NaN,70.0,NaN,Specific Gravity,All Grain,NaN,NaN,NaN,NaN,NaN
4,5,Bakke Brygg Belgisk Blonde 50 l,/homebrew/recipe/view/89534/bakke-brygg-belgis...,Belgian Blond Ale,20,50.00,1.060,1.010,6.48,17.84,...,1.050,72.0,NaN,Specific Gravity,All Grain,NaN,19.00,Sukkerlake,6-7 g sukker/l,18325.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73856,73857,Amber Alfie 2,/homebrew/recipe/view/609673/amber-alfie-2,British Strong Ale,36,21.00,1.052,1.012,5.22,39.73,...,1.060,63.0,3.0,Specific Gravity,All Grain,1.25,20.00,NaN,NaN,59658.0
73857,73858,Rye IPA,/homebrew/recipe/view/610955/rye-ipa,Specialty IPA: Rye IPA,151,10.00,1.059,1.010,6.37,59.21,...,1.034,60.0,NaN,Specific Gravity,BIAB,0.50,22.00,NaN,NaN,NaN
73858,73859,SK-Kölsch,/homebrew/recipe/view/586891/,NaN,111,16.00,1.046,1.008,5.01,25.42,...,1.041,65.0,NaN,Specific Gravity,BIAB,0.35,17.00,sucrose,140 g,82450.0
73859,73860,Flata Rødkløver,/homebrew/recipe/view/603788/,Irish Red Ale,92,24.00,1.051,1.014,4.82,25.65,...,1.043,72.0,NaN,Specific Gravity,All Grain,NaN,18.00,Sukkerlake,5 g sukker/l,NaN


In [3]:
#Eliminaçao das colunas sem dados numericos ou dados que nao sao importantes
dados1=dados.drop(['BeerID','Name','URL','Style','SugarScale','PrimingMethod','PrimingAmount'],axis=1)
#Convertendo uma coluna de dados de texto para numeros
brewmet_convert=pd.get_dummies(dados1['BrewMethod'])
dados1.drop('BrewMethod', axis=1, inplace=True)
dados1=pd.concat([dados1, brewmet_convert], axis=1)
#Eliminacao das colunas que possuem muitos dados faltantes
dados1.drop(['MashThickness','PitchRate','PrimaryTemp','UserId'],axis=1, inplace=True)
#PREENCHIMENTO DE DADOS FALTANTES DA MANEIRA TRADICIONAL
dados1['BoilGravity'].fillna(dados1['BoilGravity'].median(), inplace=True)
#Utilizar somente classes com mais de 1000 amostras
lista=dados1['StyleID'].value_counts()
lista1=lista.loc[lista<1000]
numlinhas=lista1.count()
numlinhas
l=0
m=0
for k in range (numlinhas):
    l=lista1.index[k]
    m=lista1[l]
    for i in range(m):
        dados2=dados1.loc[dados1['StyleID']!=l]
        dados1=dados2
#Separaçao dos dados entre variaveis(x) e target(y)
dadosx=dados2.drop('StyleID', axis=1)
dadosy=dados2['StyleID']

In [4]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
modelo=AdaBoostClassifier()
divdados=StratifiedKFold(n_splits=3,shuffle=True)
resultado=cross_val_score(modelo,dadosx,dadosy,cv=divdados)

In [5]:
resultado.mean()

0.5314476061427281

In [6]:
from sklearn.model_selection import GridSearchCV
modelo=AdaBoostClassifier(n_estimators=500)
param={'learning_rate':[0.2,0.1,0.05]}
otimiza=GridSearchCV(modelo,param,n_jobs=-1)
otimiza.fit(dadosx,dadosy)

GridSearchCV(estimator=AdaBoostClassifier(n_estimators=500), n_jobs=-1,
             param_grid={'learning_rate': [0.2, 0.1, 0.05]})

In [7]:
otimiza.best_score_

0.5672141420605721

In [8]:
otimiza.best_params_

{'learning_rate': 0.1}